## Preprocessing



In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import files
import tensorflow as tf
import pandas as pd 

In [22]:
# Import and read the charity_data.csv.
videogame_df = pd.read_csv("final_videoGameSales_df.csv")
videogame_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006-01-01,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985-01-01,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008-01-01,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009-01-01,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996-01-01,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
#videogame_df_renamed_columns = videogame_df.rename(columns = {'NA_Sales': 'North America', 'EU_Sales':'EU', 'JP_Sales':'Japan', 'Other_Sales':'Other', 'Global_Sales':'Global'})
#videogame_df_renamed_columns

,Rank,Name,Platform,Year,Genre,Publisher,North America,EU,Japan,Other,Global
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [25]:
# drop columns
videogame_df_dropped = videogame_df.drop(columns=['Rank', 'Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'])
videogame_df_dropped

,Platform,Year,Genre,Publisher,Global_Sales
0,Wii,2006-01-01,Sports,Nintendo,82.74
1,NES,1985-01-01,Platform,Nintendo,40.24
2,Wii,2008-01-01,Racing,Nintendo,35.82
3,Wii,2009-01-01,Sports,Nintendo,33.00
4,GB,1996-01-01,Role-Playing,Nintendo,31.37
...,...,...,...,...,...
16286,GBA,2002-01-01,Platform,Kemco,0.01
16287,GC,2003-01-01,Shooter,Infogrames,0.01
16288,PS2,2008-01-01,Racing,Activision,0.01
16289,DS,2010-01-01,Puzzle,7G//AMES,0.01


In [6]:
# Unpivot Sales Columns
#videogame_df_unpivoted = pd.melt(videogame_df_dropped, id_vars=['Platform','Year','Genre','Publisher'], var_name = 'Region', value_name = 'Sales')
#videogame_df_unpivoted

,Platform,Year,Genre,Publisher,Region,Sales
0,Wii,2006.0,Sports,Nintendo,North America,41.49
1,NES,1985.0,Platform,Nintendo,North America,29.08
2,Wii,2008.0,Racing,Nintendo,North America,15.85
3,Wii,2009.0,Sports,Nintendo,North America,15.75
4,GB,1996.0,Role-Playing,Nintendo,North America,11.27
...,...,...,...,...,...,...
66387,GBA,2002.0,Platform,Kemco,Other,0.00
66388,GC,2003.0,Shooter,Infogrames,Other,0.00
66389,PS2,2008.0,Racing,Activision,Other,0.00
66390,DS,2010.0,Puzzle,7G//AMES,Other,0.00


In [26]:
# Determine the number of unique values in each column.
unique_cols = videogame_df_dropped.nunique()
unique_cols

Platform         31
Year             39
Genre            12
Publisher       576
Global_Sales    621
dtype: int64

In [27]:
# Look at publisher counts for binning
publisher_val_counts = videogame_df_dropped['Publisher'].value_counts()
publisher_val_counts

Electronic Arts                 1339
Activision                       966
Namco Bandai Games               928
Ubisoft                          918
Konami Digital Entertainment     823
                                ... 
Detn8 Games                        1
Pow                                1
Navarre Corp                       1
MediaQuest                         1
UIG Entertainment                  1
Name: Publisher, Length: 576, dtype: int64

In [28]:
# Choose a cutoff value and create a list of names_replace to be replaced
# use the variable name names_replace
publisher_replace = list(publisher_val_counts[publisher_val_counts<300].index)
publisher_replace

# Replace in dataframe
for name in publisher_replace:
    videogame_df_dropped['Publisher'] = videogame_df_dropped['Publisher'].replace(name,"Other")

# Check to make sure binning was successful
videogame_df_dropped['Publisher'].value_counts()

Other                           7122
Electronic Arts                 1339
Activision                       966
Namco Bandai Games               928
Ubisoft                          918
Konami Digital Entertainment     823
THQ                              712
Nintendo                         696
Sony Computer Entertainment      682
Sega                             632
Take-Two Interactive             412
Capcom                           376
Atari                            347
Tecmo Koei                       338
Name: Publisher, dtype: int64

In [29]:
videogame_df_dropped

,Platform,Year,Genre,Publisher,Global_Sales
0,Wii,2006-01-01,Sports,Nintendo,82.74
1,NES,1985-01-01,Platform,Nintendo,40.24
2,Wii,2008-01-01,Racing,Nintendo,35.82
3,Wii,2009-01-01,Sports,Nintendo,33.00
4,GB,1996-01-01,Role-Playing,Nintendo,31.37
...,...,...,...,...,...
16286,GBA,2002-01-01,Platform,Other,0.01
16287,GC,2003-01-01,Shooter,Other,0.01
16288,PS2,2008-01-01,Racing,Activision,0.01
16289,DS,2010-01-01,Puzzle,Other,0.01


In [30]:
# Look at Sales counts for binning
sales_val_counts = videogame_df_dropped['Global_Sales'].value_counts()
sales_val_counts

0.02    1045
0.03     795
0.04     634
0.05     624
0.01     600
        ... 
4.98       1
5.01       1
5.05       1
5.07       1
3.16       1
Name: Global_Sales, Length: 621, dtype: int64

In [31]:
# New column that identifies if a game sales is greater than $100 000 in a given region
videogame_df_dropped.loc[videogame_df_dropped['Global_Sales'] <= 0.1, 'hit_or_miss'] = 0 
videogame_df_dropped.loc[videogame_df_dropped['Global_Sales'] > 0.1, 'hit_or_miss'] = 1
videogame_df_dropped

,Platform,Year,Genre,Publisher,Global_Sales,hit_or_miss
0,Wii,2006-01-01,Sports,Nintendo,82.74,1.0
1,NES,1985-01-01,Platform,Nintendo,40.24,1.0
2,Wii,2008-01-01,Racing,Nintendo,35.82,1.0
3,Wii,2009-01-01,Sports,Nintendo,33.00,1.0
4,GB,1996-01-01,Role-Playing,Nintendo,31.37,1.0
...,...,...,...,...,...,...
16286,GBA,2002-01-01,Platform,Other,0.01,0.0
16287,GC,2003-01-01,Shooter,Other,0.01,0.0
16288,PS2,2008-01-01,Racing,Activision,0.01,0.0
16289,DS,2010-01-01,Puzzle,Other,0.01,0.0


In [32]:
# Convert categorical data to numeric with `pd.get_dummies`
final_videogame_df = pd.get_dummies(videogame_df_dropped)
final_videogame_df

,Global_Sales,hit_or_miss,Platform_2600,Platform_3DO,Platform_3DS,Platform_DC,Platform_DS,Platform_GB,Platform_GBA,Platform_GC,...,Publisher_Konami Digital Entertainment,Publisher_Namco Bandai Games,Publisher_Nintendo,Publisher_Other,Publisher_Sega,Publisher_Sony Computer Entertainment,Publisher_THQ,Publisher_Take-Two Interactive,Publisher_Tecmo Koei,Publisher_Ubisoft
0,82.74,1.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,40.24,1.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,35.82,1.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,33.00,1.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,31.37,1.0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16286,0.01,0.0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
16287,0.01,0.0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
16288,0.01,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16289,0.01,0.0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Compile, Train and Evaluate the Model

In [41]:
# Split our preprocessed data into our features and target arrays
X = final_videogame_df.drop(['hit_or_miss'], axis=1)
y = final_videogame_df['hit_or_miss']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
# Obtain the training shape
X_train_scaled.shape

(12218, 97)

In [44]:
# Obtain the training input features
input_features = X_train_scaled.shape[1]

## Automated Neural Network Optimization

In [45]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model_op2 = tf.keras.models.Sequential()

    # Allow keras-tuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow keras-tuner to decide number of neurons in first layer
    nn_model_op2.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value= 90,
        step=5), activation=activation, input_dim=input_features))

    # Allow keras-tuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn_model_op2.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model_op2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model_op2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model_op2

In [46]:
# Import the keras-tuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [47]:
# Run the keras-tuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))


Search: Running Trial #70

Value             |Best Value So Far |Hyperparameter
sigmoid           |sigmoid           |activation
61                |61                |first_units
4                 |4                 |num_layers
11                |6                 |units_0
26                |11                |units_1
11                |1                 |units_2
16                |26                |units_3
17                |2                 |tuner/epochs
6                 |0                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
1                 |0                 |tuner/round
0054              |None              |tuner/trial_id



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/tuners/hyperband.py", line 425, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 213, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 155, in _try_build
    model = sel

RuntimeError: ignored

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters()[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models()[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export and save our model to HDF5 file
best_model.save('/content/Videogame_Optimization_735.h5')
files.download('/content/Videogame_Optimization_735.h5')